<a href="https://colab.research.google.com/github/Engineer-Ayesha-Shafique/Brain-Tumor-Segmentation-and-Detection-using-UNET-and-Watershed-in-Python/blob/main/Brain_Tumor_Segmentation_and_Detection_using_UNET_and_Watershed_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive

drive.mount('/content/drive' ,force_remount=True)

Mounted at /content/drive


#UNET



In [4]:
!pip install tensorflow
!pip install --quiet segmentation-models tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [5]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

import tensorflow as tf
import keras
import segmentation_models as sm
from segmentation_models import Unet, losses, metrics
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (ModelCheckpoint, CSVLogger,
                                        ReduceLROnPlateau, EarlyStopping,
                                        TensorBoard)
from segmentation_models import (
    Unet, FPN, Linknet, PSPNet,
    losses, metrics
)

# Tell segmentation-models to use tf.keras as its backend
sm.set_framework('tf.keras')
sm.framework()



import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv2D, BatchNormalization, Activation,
    MaxPool2D, UpSampling2D, Concatenate,
    GlobalAveragePooling2D, Reshape, Multiply, Add
)
from tensorflow.keras.models import Model
def build_dgnet_with_unet(backbone='efficientnetb7', input_shape=(None,None,3), n_classes=5):
    """
    Uses segmentation_models.Unet under the hood.
    - backbone: any SM backbone string
    - input_shape: set to your dynamic or fixed H,W,3
    - n_classes: number of output classes
    """
    # builds a Unet model with pretrained ImageNet encoder
    model = Unet(
        backbone_name=backbone,
        input_shape=input_shape,
        classes=n_classes,
        activation='softmax',
        encoder_weights='imagenet',
    )
    return model


    return Model(inp, out, name="DGNet")
# ─── Usage ────────────────────────────────────────────────────────────
NUM_CLASSES  = 5
INPUT_SHAPE  = (224, 224, 3)
model = build_dgnet_with_unet('efficientnetb7',INPUT_SHAPE, NUM_CLASSES)
model.summary()


Segmentation Models: using `tf.keras` framework.
258434480/258434480 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │      1,728 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        256 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        576 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        256 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 64)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 64)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 16)  │      1,040 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 64)  │      1,088 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 64)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │      2,048 │ block1a_se_excit… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 112, 112,  │        128 │ block1a_project_… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_dwconv      │ (None, 112, 112,  │        288 │ block1a_project_… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_bn          │ (None, 112, 112,  │        128 │ block1b_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_activation  │ (None, 112, 112,  │          0 │ block1b_bn[0][0]

 Total params: 75,048,677 (286.29 MB)

 Trainable params: 74,735,973 (285.10 MB)

 Non-trainable params: 312,704 (1.19 MB)

In [6]:
# install visualkeras for model plotting
!pip install --quiet visualkeras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.4/997.4 kB 17.3 MB/s eta 0:00:00


# Metrics

In [7]:
import tensorflow as tf

smooth = 1e-15
num_classes = 5  # <-- make this match your final Conv2D(filters=5, ...)

def dice_coefficient(y_true, y_pred):
    # y_true: (B, H, W, 1) integers 0..4
    # y_pred: (B, H, W, 5) softmax probs

    # 1) Squeeze off the last dim → (B, H, W)
    y_true = tf.squeeze(y_true, axis=-1)

    # 2) One‑hot encode → (B, H, W, 5)
    y_true_o = tf.one_hot(tf.cast(y_true, tf.int32), depth=num_classes)

    # 3) Flatten both to (B*H*W, 5)
    y_true_f = tf.reshape(y_true_o, [-1, num_classes])
    y_pred_f = tf.reshape(y_pred,   [-1, num_classes])

    # 4) Compute intersection & union per class
    intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=0)
    union        = tf.reduce_sum(y_true_f, axis=0) + tf.reduce_sum(y_pred_f, axis=0)

    # 5) Dice per class, then average
    dice_per_class = (2. * intersection + smooth) / (union + smooth)
    return tf.reduce_mean(dice_per_class)

def dice_coefficient_loss(y_true, y_pred):
    return 1.0 - dice_coefficient(y_true, y_pred)


#Train


In [8]:
!pip install --quiet tensorflow-io

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 45.8 MB/s eta 0:00:00


In [9]:
# In a notebook cell, before any cv2 imports:
!pip install --quiet opencv-python-headless
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# 1) Import TF first and set memory growth (fallback to experimental)
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        try:
            tf.config.experimental.set_memory_growth(gpu, True)
        except Exception as e:
            print(f"[GPU] Could not set memory growth: {e}")
    print(f"[GPU] Detected {len(gpus)} GPU(s).")

# 2) Now import the rest
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import (
    ModelCheckpoint, CSVLogger, ReduceLROnPlateau,
    EarlyStopping, TensorBoard
)
from tensorflow.keras.optimizers import Adam
import tensorflow_io as tfio

""" Global parameters """
Height = 224
Width  = 224

def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_dataset(path_dataset, val_ratio=0.2, test_ratio=0.1, random_state=42):
    img_dir  = os.path.join(path_dataset, "images")
    mask_dir = os.path.join(path_dataset, "masks")

    image_exts = ('.png', '.jpg', '.jpeg')
    images = sorted([
        os.path.join(img_dir, f)
        for f in os.listdir(img_dir)
        if f.lower().endswith(image_exts)
    ])


    mask_exts = ('.tif', '.tiff')
    masks = sorted([
        os.path.join(mask_dir, f)
        for f in os.listdir(mask_dir)
        if f.lower().endswith(mask_exts)
    ])

    if not images:
        raise RuntimeError(f"No images found in {img_dir}")
    if not masks:
        raise RuntimeError(f"No masks found in {mask_dir}")

    images_lookup = {
        os.path.splitext(os.path.basename(p))[0]: p
        for p in images
    }
#MildDemented_0a7b1321-eba0-40dc-85b8-de34241554a2_jpg.rf.70370fcd9ac0fb82f91d27f558053d84.tif
    pairs = []
    for m in masks:
        base = os.path.splitext(os.path.basename(m))[0]
        if base.endswith('_mask'):
            root = base[:-5]
            img_path = images_lookup.get(root)
            if img_path:
                pairs.append((img_path, m))

    if not pairs:
        raise RuntimeError("No matching image↔mask pairs found!")

    trainval, test = train_test_split(pairs, test_size=test_ratio, random_state=random_state)
    val_frac = val_ratio / (1.0 - test_ratio)
    train, val = train_test_split(trainval, test_size=val_frac, random_state=random_state)

    train_imgs, train_masks = zip(*train)
    val_imgs,   val_masks   = zip(*val)
    test_imgs,  test_masks  = zip(*test)

    return (
        list(train_imgs), list(train_masks),
        list(val_imgs),   list(val_masks),
        list(test_imgs),  list(test_masks)
    )

def read_image_file(path_img):
    path = path_img.decode('utf-8') if isinstance(path_img, bytes) else path_img
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (Width, Height)) / 255.0
    return img.astype(np.float32)

def read_mask_file(path_mask):
    path = path_mask.decode('utf-8') if isinstance(path_mask, bytes) else path_mask
    mask = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if mask.ndim == 3:
        mask = mask[..., 0]
    mask = cv2.resize(mask, (Width, Height), interpolation=cv2.INTER_NEAREST)

    # ✅ Clip to [0, 4], not [0, 3]:
    mask = np.clip(mask, 0, NUM_CLASSES-1)

    # Convert to integer class IDs
    mask = mask.astype(np.int32)

    # Add the channel dim back
    return mask[..., None]

import cv2

def tf_parse(path_img, path_mask, img_size):
    # Decode & preprocess image (.png)
    img = tf.io.read_file(path_img)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [img_size, img_size])
    img = tf.cast(img, tf.float32) / 255.0

    # Decode & preprocess mask (supports .png or .tif via OpenCV)
    def _decode_mask_np(mask_path, size):
        import cv2, numpy as np
        # mask_path is a tf.Tensor of type string; convert to Python path
        path = mask_path.numpy().decode('utf-8')
        s    = int(size.numpy())
        # read mask (TIFF or PNG)
        mask = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        # if multi-channel, keep only first channel
        if mask is None:
            raise FileNotFoundError(f"Could not read mask at {path}")
        if mask.ndim > 2:
            mask = mask[..., 0]
        # resize with nearest‐neighbor
        mask = cv2.resize(mask, (s, s), interpolation=cv2.INTER_NEAREST)
        # expand dims to (H, W, 1)
        mask = mask.astype(np.int32)[..., np.newaxis]
        return mask

    mask = tf.py_function(
        func=_decode_mask_np,
        inp=[path_mask, img_size],
        Tout=tf.int32
    )
    # inform TensorFlow of the static shape
    mask.set_shape([img_size, img_size, 1])

    return img, mask



def tf_dataset(inputs, targets, batch_size=2, img_size=224):
    ds = tf.data.Dataset.from_tensor_slices((inputs, targets))
    ds = ds.map(lambda i, m: tf_parse(i, m, img_size),
                num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.shuffle(224).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds


def edit_mask_images(mask_dir, func, output_dir=None):
    out_dir = output_dir or mask_dir
    create_folder(out_dir)
    exts = ('.tif')
    for fname in os.listdir(mask_dir):
        if not fname.lower().endswith(exts):
            continue
        in_path  = os.path.join(mask_dir, fname)
        mask     = cv2.imread(in_path, cv2.IMREAD_UNCHANGED)
        edited   = func(mask)
        cv2.imwrite(os.path.join(out_dir, fname), edited)
    print(f"[edit_mask_images] Applied edits to masks in {out_dir}")

if __name__ == "__main__":
    np.random.seed(42)
    tf.random.set_seed(42)

    create_folder("/content/drive/MyDrive/Project_41725/files")

    batch_size = 16
    lr         = 1e-4
    epochs_num     = 100
    path_model = os.path.join("/content/drive/MyDrive/Project_41725/files", "model.h5")
    path_csv   = os.path.join("/content/drive/MyDrive/Project_41725/files", "log.csv")

    ds_path = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg"
    train_imgs, train_masks, val_imgs, val_masks, test_imgs, test_masks = load_dataset(ds_path)

    print(f"Train: {len(train_imgs)}, Val: {len(val_imgs)}, Test: {len(test_imgs)}")

    train_ds = tf_dataset(train_imgs, train_masks, batch_size=batch_size)
    val_ds   = tf_dataset(val_imgs,   val_masks,   batch_size=batch_size)
    test_ds  = tf_dataset(test_imgs,  test_masks,  batch_size=batch_size)

    # Build, compile and fit your model below:
    # model = Unet_model((Height, Width, 3))
    # model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coeff])
    # callbacks = [ModelCheckpoint(...), ...]
    # model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=callbacks)


[GPU] Could not set memory growth: Physical devices cannot be modified after being initialized
[GPU] Detected 1 GPU(s).


/usr/local/lib/python3.11/dist-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.11/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/usr/local/lib/python3.11/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl8str_util8EndsWithESt17basic_string_viewIcSt11char_traitsIcEES4_']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.11/dist-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.11/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.11/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZN3tsl8str_util9LowercaseB5cxx11ESt17basic_s

Train: 1216, Val: 348, Test: 174


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
)
from tensorflow.keras.models import load_model
from sklearn.metrics import f1_score, jaccard_score, recall_score, precision_score

# ─── Hyper-parameters & paths ───
NUM_CLASSES   = 5
INPUT_SHAPE   = (224, 224, 3)
EPOCHS        = 320
BATCH_SIZE    = 8                        # choose as you like
path_model    = "/content/drive/MyDrive/Project_41725/files/model.h5"
path_csv      = "/content/drive/MyDrive/Project_41725/files/log.csv"
path_score_csv= "/content/drive/MyDrive/Project_41725/files/scores.csv"

# ─── Progressive-Resizing Schedule ───
RESOLUTIONS   = [256, INPUT_SHAPE[0]]    # 256×256 then 224×224
EPOCHS_PHASE  = [160, 160]                 # split your 160 epochs
LR_PHASE      = [1e-3, 1e-4]             # high LR → global, low LR → details

# ─── Build model & losses/metrics ───
tf.keras.backend.clear_session()
model = build_dgnet_with_unet('efficientnetb7',(None,None,3), NUM_CLASSES)
model._name = "unet"

def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true = tf.squeeze(y_true, axis=-1)
    y_true_o = tf.one_hot(tf.cast(y_true, tf.int32), depth=NUM_CLASSES)
    y_true_f = tf.reshape(y_true_o, [-1, NUM_CLASSES])
    y_pred_f = tf.reshape(y_pred,   [-1, NUM_CLASSES])
    inter = tf.reduce_sum(y_true_f * y_pred_f, axis=0)
    denom = tf.reduce_sum(y_true_f, axis=0) + tf.reduce_sum(y_pred_f, axis=0)
    dice_per_class = (2. * inter + smooth) / (denom + smooth)
    return tf.reduce_mean(dice_per_class)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def combined_loss(y_true, y_pred):
    ce = tf.keras.losses.SparseCategoricalCrossentropy()(y_true, y_pred)
    return ce + dice_loss(y_true, y_pred)

class MeanIoUArgMax(tf.keras.metrics.MeanIoU):
    def __init__(self, num_classes=NUM_CLASSES, name="mean_iou", dtype=None):
        super().__init__(num_classes=num_classes, name=name, dtype=dtype)
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.squeeze(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        return super().update_state(y_true, y_pred, sample_weight)

# ─── Initial compile at small‐phase LR ───
opt = Adam(LR_PHASE[0])
model.compile(
    optimizer=opt,
    loss=combined_loss,
    metrics=[dice_coef, MeanIoUArgMax()]
)

# ─── Two-phase progressive resizing ───
# assume train_imgs, train_masks, val_imgs, val_masks, test_imgs, test_masks
# have already been created via your load_dataset(...) call
for phase, res in enumerate(RESOLUTIONS):
    print(f"\n>>> Phase {phase+1}: training at {res}×{res}")

    # rebuild your datasets at this res
    train_ds = tf_dataset(train_imgs, train_masks,
                          batch_size=BATCH_SIZE,
                          img_size=res)
    val_ds   = tf_dataset(val_imgs,   val_masks,
                          batch_size=BATCH_SIZE,
                          img_size=res)

    # freeze / unfreeze as before…
    if phase == 0:
        for layer in model.layers:
            if 'encoder' in layer.name:
                layer.trainable = False
    else:
        for layer in model.layers:
            layer.trainable = True

    # re-compile with a brand-new Adam at the desired LR
    model.compile(
        optimizer=Adam(LR_PHASE[phase]),
        loss=combined_loss,
        metrics=[dice_coef, MeanIoUArgMax()]
    )

    # callbacks
    ckpt_path = f"/content/drive/MyDrive/Project_41725/files/model_{res}.h5"
    phase_cbs = [
        ModelCheckpoint(ckpt_path, save_best_only=True, verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                          patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(path_csv, append=True),
        EarlyStopping(monitor='val_loss', patience=20)
    ]

    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS_PHASE[phase],
        callbacks=phase_cbs
    )

    model.load_weights(ckpt_path)

# ─── Save final full-resolution model ───
model.save(path_model)
print("Final model saved to", path_model)

# ─── Inference & per-image metrics ───
test_ds = tf_dataset(test_imgs, test_masks,
                     batch_size=BATCH_SIZE,
                     img_size=INPUT_SHAPE[0])

EVL_SCORE_list = []
for batch_index, (batch_images, batch_masks) in enumerate(test_ds):
    preds = model.predict(batch_images)
    for i in range(len(preds)):
        y_true = batch_masks[i].numpy().squeeze(-1)
        y_pred = np.argmax(preds[i], axis=-1)
        f1 = f1_score(y_true.flatten(), y_pred.flatten(),
                      average='macro', zero_division=0)
        j  = jaccard_score(y_true.flatten(), y_pred.flatten(),
                           average='macro', zero_division=0)
        r  = recall_score(y_true.flatten(), y_pred.flatten(),
                          average='macro', zero_division=0)
        p  = precision_score(y_true.flatten(), y_pred.flatten(),
                             average='macro', zero_division=0)
        img_name = f"batch{batch_index}_idx{i}"
        EVL_SCORE_list.append((img_name, f1, j, r, p))

# ─── Save per-image scores ───
df_scores = pd.DataFrame(
    EVL_SCORE_list,
    columns=["Image", "F1", "Jaccard", "Recall", "Precision"]
)
os.makedirs(os.path.dirname(path_score_csv), exist_ok=True)
df_scores.to_csv(path_score_csv, index=False)
print("Wrote", path_score_csv)



>>> Phase 1: training at 256×256
Epoch 1/160
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 557ms/step - dice_coef: 0.3130 - loss: 1.1940 - mean_iou: 0.3278
Epoch 1: val_loss improved from inf to 0.80069, saving model to /content/drive/MyDrive/Project_41725/files/model_256.h5


152/152 ━━━━━━━━━━━━━━━━━━━━ 462s 1s/step - dice_coef: 0.3133 - loss: 1.1921 - mean_iou: 0.3280 - val_dice_coef: 0.3656 - val_loss: 0.8007 - val_mean_iou: 0.3785 - learning_rate: 0.0010
Epoch 2/160
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - dice_coef: 0.4267 - loss: 0.6554 - mean_iou: 0.3871
Epoch 2: val_loss did not improve from 0.80069
152/152 ━━━━━━━━━━━━━━━━━━━━ 39s 236ms/step - dice_coef: 0.4268 - loss: 0.6553 - mean_iou: 0.3872 - val_dice_coef: 0.3826 - val_loss: 1.0036 - val_mean_iou: 0.3438 - learning_rate: 0.0010
Epoch 3/160
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - dice_coef: 0.4893 - loss: 0.5972 - mean_iou: 0.4729
Epoch 3: val_loss did not improve from 0.80069
152/152 ━━━━━━━━━━━━━━━━━━━━ 20s 127ms/step - dice_coef: 0.4894 - loss: 0.5971 - mean_iou: 0.4729 - val_dice_coef: 0.3697 - val_loss: 1.9262 - val_mean_iou: 0.3444 - learning_rate: 0.0010
Epoch 4/160
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - dice_coef: 0.5266 - loss: 0.5496 - mean_iou: 0.5048
Epoch 4: val_loss

152/152 ━━━━━━━━━━━━━━━━━━━━ 34s 216ms/step - dice_coef: 0.5266 - loss: 0.5496 - mean_iou: 0.5049 - val_dice_coef: 0.5574 - val_loss: 0.5048 - val_mean_iou: 0.5408 - learning_rate: 0.0010
Epoch 5/160
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step - dice_coef: 0.5726 - loss: 0.4965 - mean_iou: 0.5528
Epoch 5: val_loss improved from 0.50477 to 0.49681, saving model to /content/drive/MyDrive/Project_41725/files/model_256.h5


152/152 ━━━━━━━━━━━━━━━━━━━━ 45s 274ms/step - dice_coef: 0.5726 - loss: 0.4964 - mean_iou: 0.5529 - val_dice_coef: 0.5738 - val_loss: 0.4968 - val_mean_iou: 0.5634 - learning_rate: 0.0010
Epoch 6/160
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - dice_coef: 0.6072 - loss: 0.4476 - mean_iou: 0.5959
Epoch 6: val_loss improved from 0.49681 to 0.43211, saving model to /content/drive/MyDrive/Project_41725/files/model_256.h5


152/152 ━━━━━━━━━━━━━━━━━━━━ 43s 280ms/step - dice_coef: 0.6073 - loss: 0.4474 - mean_iou: 0.5960 - val_dice_coef: 0.6217 - val_loss: 0.4321 - val_mean_iou: 0.6122 - learning_rate: 0.0010
Epoch 7/160
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - dice_coef: 0.6671 - loss: 0.3932 - mean_iou: 0.6685
Epoch 7: val_loss did not improve from 0.43211
152/152 ━━━━━━━━━━━━━━━━━━━━ 39s 238ms/step - dice_coef: 0.6671 - loss: 0.3933 - mean_iou: 0.6685 - val_dice_coef: 0.5472 - val_loss: 0.5712 - val_mean_iou: 0.5307 - learning_rate: 0.0010
Epoch 8/160
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - dice_coef: 0.6828 - loss: 0.3810 - mean_iou: 0.6682
Epoch 8: val_loss improved from 0.43211 to 0.35863, saving model to /content/drive/MyDrive/Project_41725/files/model_256.h5


...

152/152 ━━━━━━━━━━━━━━━━━━━━ 32s 202ms/step - dice_coef: 0.8543 - loss: 0.1685 - mean_iou: 0.9162 - val_dice_coef: 0.8052 - val_loss: 0.2354 - val_mean_iou: 0.8207 - learning_rate: 1.0000e-05
Epoch 28/160
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - dice_coef: 0.8322 - loss: 0.1853 - mean_iou: 0.9143
Epoch 28: val_loss did not improve from 0.23542
152/152 ━━━━━━━━━━━━━━━━━━━━ 39s 239ms/step - dice_coef: 0.8321 - loss: 0.1854 - mean_iou: 0.9141 - val_dice_coef: 0.7927 - val_loss: 0.2473 - val_mean_iou: 0.8211 - learning_rate: 1.0000e-05
Epoch 29/160
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - dice_coef: 0.8077 - loss: 0.2120 - mean_iou: 0.9154
Epoch 29: val_loss did not improve from 0.23542
152/152 ━━━━━━━━━━━━━━━━━━━━ 20s 125ms/step - dice_coef: 0.8077 - loss: 0.2120 - mean_iou: 0.9152 - val_dice_coef: 0.7814 - val_loss: 0.2603 - val_mean_iou: 0.8204 - learning_rate: 1.0000e-05
Epoch 30/160
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - dice_coef: 0.8296 - loss: 0.1920 - mean_iou: 0.917

152/152 ━━━━━━━━━━━━━━━━━━━━ 333s 387ms/step - dice_coef: 0.8142 - loss: 0.2069 - mean_iou: 0.8812 - val_dice_coef: 0.7987 - val_loss: 0.2391 - val_mean_iou: 0.8278 - learning_rate: 1.0000e-04
Epoch 2/160
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - dice_coef: 0.8374 - loss: 0.1845 - mean_iou: 0.8954
Epoch 2: val_loss did not improve from 0.23909
152/152 ━━━━━━━━━━━━━━━━━━━━ 22s 122ms/step - dice_coef: 0.8373 - loss: 0.1846 - mean_iou: 0.8953 - val_dice_coef: 0.8040 - val_loss: 0.2395 - val_mean_iou: 0.8277 - learning_rate: 1.0000e-04
Epoch 3/160
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - dice_coef: 0.8242 - loss: 0.2048 - mean_iou: 0.9024
Epoch 3: val_loss did not improve from 0.23909
152/152 ━━━━━━━━━━━━━━━━━━━━ 19s 121ms/step - dice_coef: 0.8241 - loss: 0.2049 - mean_iou: 0.9022 - val_dice_coef: 0.7785 - val_loss: 0.2590 - val_mean_iou: 0.8345 - learning_rate: 1.0000e-04
Epoch 4/160
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - dice_coef: 0.8271 - loss: 0.1956 - mean_iou: 0.8986
Ep

152/152 ━━━━━━━━━━━━━━━━━━━━ 32s 202ms/step - dice_coef: 0.8210 - loss: 0.1965 - mean_iou: 0.9139 - val_dice_coef: 0.8003 - val_loss: 0.2389 - val_mean_iou: 0.8332 - learning_rate: 1.0000e-04
Epoch 6/160
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step - dice_coef: 0.8206 - loss: 0.1967 - mean_iou: 0.9055
Epoch 6: val_loss did not improve from 0.23887
152/152 ━━━━━━━━━━━━━━━━━━━━ 39s 235ms/step - dice_coef: 0.8206 - loss: 0.1968 - mean_iou: 0.9054 - val_dice_coef: 0.7792 - val_loss: 0.2645 - val_mean_iou: 0.8365 - learning_rate: 1.0000e-04
Epoch 7/160
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - dice_coef: 0.8059 - loss: 0.2173 - mean_iou: 0.9047
Epoch 7: val_loss did not improve from 0.23887
152/152 ━━━━━━━━━━━━━━━━━━━━ 19s 120ms/step - dice_coef: 0.8059 - loss: 0.2174 - mean_iou: 0.9046 - val_dice_coef: 0.7772 - val_loss: 0.2599 - val_mean_iou: 0.8371 - learning_rate: 1.0000e-04
Epoch 8/160
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - dice_coef: 0.8284 - loss: 0.1889 - mean_iou: 0.9067
Epo

152/152 ━━━━━━━━━━━━━━━━━━━━ 33s 214ms/step - dice_coef: 0.8403 - loss: 0.1805 - mean_iou: 0.9126 - val_dice_coef: 0.8225 - val_loss: 0.2197 - val_mean_iou: 0.8396 - learning_rate: 1.0000e-05
Epoch 15/160
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step - dice_coef: 0.8257 - loss: 0.1927 - mean_iou: 0.9150
Epoch 15: val_loss did not improve from 0.21968
152/152 ━━━━━━━━━━━━━━━━━━━━ 36s 234ms/step - dice_coef: 0.8257 - loss: 0.1927 - mean_iou: 0.9149 - val_dice_coef: 0.8207 - val_loss: 0.2219 - val_mean_iou: 0.8405 - learning_rate: 1.0000e-05
Epoch 16/160
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - dice_coef: 0.8312 - loss: 0.1852 - mean_iou: 0.9177
Epoch 16: val_loss did not improve from 0.21968
152/152 ━━━━━━━━━━━━━━━━━━━━ 19s 120ms/step - dice_coef: 0.8312 - loss: 0.1852 - mean_iou: 0.9176 - val_dice_coef: 0.7979 - val_loss: 0.2432 - val_mean_iou: 0.8381 - learning_rate: 1.0000e-05
Epoch 17/160
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - dice_coef: 0.8411 - loss: 0.1757 - mean_iou: 0.923

Final model saved to /content/drive/MyDrive/Project_41725/files/model.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step


Dataset complexity & size
– If your tumors occupy tiny regions or vary wildly in appearance, 256 px may not capture enough detail to learn them well. Phase 2 at full 224×224 resolution will help, but you still need enough diverse examples to generalize.
– If you only have ~1 300 training masks (as your numbers suggest), 90% dice is ambitious without strong augmentation or external data.

Model capacity & pretraining
– Your custom R2AttU-Net is powerful, but it’s trained from scratch. If you swap in a backbone pretrained on ImageNet (e.g. ResNet34 or EfficientNet encoder via segmentation_models), you often see a big jump in initial convergence and final accuracy.

Augmentation & regularization
– Aggressive, on-the-fly augmentations (rotations, elastic deformations, intensity scaling) will help the model learn invariances and avoid overfitting on only ~1 200 examples.
– Techniques like MixUp, CutMix, or DropBlock in the decoder can further boost robustness.

Loss weighting & scheduling
– You’re using a simple sum of cross-entropy + dice loss. Tuning the relative weight (e.g. 0.3*CE + 0.7*dice) can steer training toward better boundary accuracy.
– An exponential or cyclical learning-rate schedule in Phase 2 often helps squeeze out those last few percent.

Progressive-resizing fine-tuning
– After your two phases, add a Phase 3 at full resolution but with frozen decoder (only fine-tune the decoder head) or vice versa—sometimes unfreezing in stages yields that final push above 0.90.

#Test


In [ ]:
from google.colab import drive
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab import files

# 1) Csv
import pandas as pd

# suppose you build EVL_SCORE_list as [(img_name, f1, jaccard, recall, precision), …]
df = pd.DataFrame(
    EVL_SCORE_list,
    columns=["Image", "F1", "Jaccard", "Recall", "Precision"]
)
df.to_csv(
    '/content/drive/MyDrive/DSP Project/files/score.csv',
    index=False
)
model_path = "/content/drive/MyDrive/Project_41725/files/LinkNet/model.h5"

# 3) Load model for inference only—skip compile to avoid the TypeError
model = load_model(
    model_path,
    custom_objects={
        'dice_coefficient': dice_coefficient,
        'dice_coefficient_loss': dice_coefficient_loss
    },
    compile=False
)

def predict_and_show(img_path, model, input_size=(224,224)):
    gray    = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    resized = cv2.resize(gray, input_size)
    x       = np.stack([resized]*3, -1).astype(np.float32) / 255.0

    # Run model
    pred = model.predict(x[None])[0]         # ➔ (H, W, num_classes)
    seg  = pred.argmax(-1).astype(np.uint8)  # ➔ (H, W)

    # Compute and print class percentages
    unique, counts = np.unique(seg, return_counts=True)
    total = seg.size
    percentages = {int(cls): float(cnt/total*100) for cls, cnt in zip(unique, counts)}
    print(f"\nClass percentages for {img_path}:")
    for cls, pct in percentages.items():
        print(f"  Class {cls}: {pct:.2f}%")

    # Display images
    plt.figure(figsize=(8,4))
    plt.subplot(1,2,1)
    plt.title("Input")
    plt.imshow(resized, cmap='gray')
    plt.axis('off')

    plt.subplot(1,2,2)
    plt.title("Segmentation Overlay")
    plt.imshow(resized, cmap='gray', alpha=0.5)
    plt.imshow(seg, cmap='jet',   alpha=0.5)
    plt.axis('off')
    plt.show()

# 4) Upload & run on uploaded files
uploaded = files.upload()
for fn in uploaded:
    predict_and_show(fn, model)
    print("Num output channels:", model.output_shape[-1])


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# -- ensure output folder exists --
out_dir = '/content/drive/MyDrive/DSP Project/files'
os.makedirs(out_dir, exist_ok=True)

# 1) Load evaluation results
df = pd.read_csv(os.path.join(out_dir, 'score.csv'))

# 2) (Optional) Inspect to confirm non-zero values
print(df.head(), '\n', df.dtypes)

# 3) Your plots…
plt.figure(figsize=(10,6))
for m in ['F1','Jaccard','Recall','Precision']:
    plt.hist(df[m], bins=20, alpha=0.5, label=m)
plt.legend(); plt.show()

# 4) Compute & save summary stats
summary = pd.DataFrame({
    'Metric':  ['F1','Jaccard','Recall','Precision'],
    'Mean':    df[['F1','Jaccard','Recall','Precision']].mean().values,
    'Std Dev': df[['F1','Jaccard','Recall','Precision']].std().values
})
summary_path = os.path.join(out_dir, 'summary.csv')
summary.to_csv(summary_path, index=False)
print(f"Saved summary to {summary_path}:\n", summary)


#Plot and Checks

In [ ]:
!pip install ace_tools
!pip install ace-tools

In [ ]:
import os
import glob
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from ace_tools import display_dataframe_to_user

# 1) Point to your mask directory
mask_dir = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg/masks"  # <— update this path

# 2) Collect all mask file paths
mask_paths = glob.glob(os.path.join(mask_dir, "*"))

# 3) Initialize pixel count dictionary
num_classes = 5  # update if different
pixel_counts = {cls: 0 for cls in range(num_classes)}

# 4) Count pixels per class across all masks
for path in mask_paths:
    mask = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    # If mask has channel dimension, reduce to single channel
    if mask.ndim == 3:
        mask = mask[..., 0]
    flat = mask.flatten()
    for cls in range(num_classes):
        pixel_counts[cls] += np.sum(flat == cls)

# 5) Compute total and percentages
total_pixels = sum(pixel_counts.values())
percentages = {cls: (count / total_pixels * 100) for cls, count in pixel_counts.items()}

# 6) Prepare DataFrame
df = pd.DataFrame({
    "Class": list(pixel_counts.keys()),
    "Pixel Count": list(pixel_counts.values()),
    "Percentage (%)": [percentages[cls] for cls in pixel_counts]
})

# 7) Display table to user
display_dataframe_to_user("Class Distribution", df)

# 8) Plot percentages
plt.figure(figsize=(8, 5))
plt.bar(df["Class"].astype(str), df["Percentage (%)"])
plt.title("Mask Pixel Percentage by Class")
plt.xlabel("Class")
plt.ylabel("Percentage of Total Mask Pixels")
plt.show()
